<a href="https://colab.research.google.com/github/andyradz/aplios.solutions/blob/master/introduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AR-DiplomaWorkspace

#### MOUNT GOOGLE DRIVE VOLUME

In [4]:
## MOUNT GOOGLE DRIVE VOLUME

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### LIST GOOGLE DRIVE VOLUME

In [5]:
## LIST GOOGLE DRIVE VOLUME
!ls -laFtA --full-time /content/gdrive 

### CREATE REQUIRED DATA FOLDERS

!mkdir input
!mkdir output

total 16
dr-x------  2 root root 4096 2022-11-23 13:40:59.000000000 +0000 .file-revisions-by-id/
drwx------ 22 root root 4096 2022-11-23 13:40:59.000000000 +0000 MyDrive/
dr-x------  2 root root 4096 2022-11-23 13:40:59.000000000 +0000 .shortcut-targets-by-id/
drwx------  5 root root 4096 2022-11-23 13:40:59.000000000 +0000 .Trash-0/


In [6]:
### DOWNLOAD AND INSTALL JAVA VIRTUAL MACHINE(JVM)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [7]:
### DOWNLOAD APACHE SPARK AND HADOOP ENVIRONMENT
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz > /dev/null

### UNZIP SPARK ENVIRONMENT
!tar xf spark-3.3.1-bin-hadoop3.tgz > /dev/null

--2022-11-23 13:42:01--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M   230MB/s    in 1.2s    

2022-11-23 13:42:02 (230 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [8]:
### SET SYSTEM VARIABLES FOR PYSPARK

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.1-bin-hadoop3'
os.environ['PYSPARK_SUBMIT_ARGS'] = '---jars /content/spark-3.3.1-bin-hadoop3/jars/spark-xml_2.12-0.11.0.jar pyspark-shell'

In [9]:
### INSTALL PYSPARK ENVIRONMENT
!pip install -q pyspark > /dev/null

### INSTALL LIBRARY FIND PYSPARK
!pip install -q findspark > /dev/null

### INITIAL PYSPARK ENVIRONMENT
import findspark
findspark.init()

### CHECK PYSPARK LOCATION
findspark.find()


'/content/spark-3.3.1-bin-hadoop3'

In [15]:
!sudo apt install --assume-yes p7zip-full

!type(7z)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/bin/bash: -c: line 0: syntax error near unexpected token `7z'
/bin/bash: -c: line 0: `type(7z)'


In [10]:
### LIST PYTHON INSTALLED PACKAGES

!pip3 list -v


Package                       Version                Location                               Installer
----------------------------- ---------------------- -------------------------------------- ---------
absl-py                       1.3.0                  /usr/local/lib/python3.7/dist-packages pip
aeppl                         0.0.33                 /usr/local/lib/python3.7/dist-packages pip
aesara                        2.7.9                  /usr/local/lib/python3.7/dist-packages pip
aiohttp                       3.8.3                  /usr/local/lib/python3.7/dist-packages pip
aiosignal                     1.3.1                  /usr/local/lib/python3.7/dist-packages pip
alabaster                     0.7.12                 /usr/local/lib/python3.7/dist-packages pip
albumentations                1.2.1                  /usr/local/lib/python3.7/dist-packages pip
altair                        4.2.0                  /usr/local/lib/python3.7/dist-packages pip
appdirs                     

# Download stackexchange data 

In [24]:
#!FILE=/content/input/books.xml
#!if ![-f "$FILE"]; then
#!    echo Start downloading file $FILE
#!    wget https://github.com/databricks/spark-xml/raw/master/src/test/resources/books.xml  -P /content/input/ > /dev/null
#!fi


!wget https://archive.org/download/stackexchange/mathematica.stackexchange.com.7z -P /content/input/ > /dev/null

!7za e -y /content/input/mathematica.stackexchange.com.7z -o/content/input/


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/input/                         1 file, 265738334 bytes (254 MiB)

Extracting archive: /content/input/mathematica.stackexchange.com.7z
--
Path = /content/input/mathematica.stackexchange.com.7z
Type = 7z
Physical Size = 265738334
Headers Size = 344
Method = BZip2
Solid = +
Blocks = 7

  0%      0% - Badges.xml                   1% 1 - Comments.xml                       2% 1 - Comments.xml                       3% 1 - Comments.xml                       4% 1 - Comments.xml                     

#### LOAD AND SAVE POSTS.XML DATASET TO PARQUET FORMAT

In [29]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, BooleanType, DateType 

spark = SparkSession.builder.master("local[4]").appName('SparkByExamples.com').config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.6.0').getOrCreate()


schema = StructType([ \
    StructField("_AnswerCount", LongType(), True), \
    StructField("_Body", StringType(), True), \
    StructField("_CommentCount", LongType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_LastActivityDate", StringType(), True), \
    StructField("_OwnerUserId", LongType(), True), \
    StructField("_ParentId", LongType(), True), \
    StructField("_PostTypeId", LongType(), True), \
    StructField("_Score", LongType(), True), \
    StructField("_Tags", StringType(), True), \
    StructField("_Title", StringType(), True), \
    StructField("_ViewCount", LongType(), True) 
]) 

postsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='posts') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Posts.xml')

postsDF.printSchema()          

postsDF = postsDF \
                .withColumnRenamed("_AnswerCount", "AnswerCount") \
                .withColumnRenamed("_Body", "Body") \
                .withColumnRenamed("_CommentCount", "CommentCount") \
                .withColumnRenamed("_CreationDate", "CreationDate") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_LastActivityDate", "LastActivityDate") \
                .withColumnRenamed("_OwnerUserId", "OwnerUserId") \
                .withColumnRenamed("_ParentId", "ParentId") \
                .withColumnRenamed("_PostTypeId", "PostTypeId") \
                .withColumnRenamed("_Score", "Score") \
                .withColumnRenamed("_Tags", "Tags") \
                .withColumnRenamed("_Title", "Title") \
                .withColumnRenamed("_ViewCount", "ViewCount")

postsDF.show(n=20, truncate=True)

postsDF \
      .write \
      .partitionBy("PostTypeId") \
      .mode("overwrite") \
      .parquet("/content/output/posts.parquet")

root
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CreationDate: string (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)

+-----------+--------------------+------------+--------------------+---+--------------------+-----------+--------+----------+-----+--------------------+--------------------+---------+
|AnswerCount|                Body|CommentCount|        CreationDate| Id|    LastActivityDate|OwnerUserId|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+-----------+--------------------+------------+--------------------+---+--------------------+-----------

#### LOAD AND SAVE BADGES.XML DATASET TO PARQUET FORMAT

In [32]:
schema = StructType([ \
    StructField("_Class", LongType(), True), \
    StructField("_Date", StringType(), True), \
    StructField("_CommentCount", LongType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_Name", StringType(), True), \
    StructField("_TagBased", BooleanType(), True), \
    StructField("_UserId", LongType(), True)    
])

badgesDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='badges') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Badges.xml')

badgesDF = badgesDF \
                .withColumnRenamed("_Class", "Class") \
                .withColumnRenamed("_Date", "Date") \
                .withColumnRenamed("_CommentCount", "CommentCount") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_Name", "Name") \
                .withColumnRenamed("_TagBased", "TagBased") \
                .withColumnRenamed("_UserId", "UserId")

badgesDF.show()

badgesDF \
      .write \
      .partitionBy("Class") \
      .mode("overwrite") \
      .parquet("/content/output/badges.parquet")

+-----+--------------------+------------+---+--------------+--------+------+
|Class|                Date|CommentCount| Id|          Name|TagBased|UserId|
+-----+--------------------+------------+---+--------------+--------+------+
|    3|2012-01-17T20:56:...|        null|  1|Autobiographer|   false|     1|
|    3|2012-01-17T20:56:...|        null|  2|Autobiographer|   false|     2|
|    3|2012-01-17T20:56:...|        null|  3|Autobiographer|   false|     3|
|    3|2012-01-17T21:01:...|        null|  4|Autobiographer|   false|     8|
|    3|2012-01-17T21:06:...|        null|  5|    Analytical|   false|     7|
|    3|2012-01-17T21:06:...|        null|  6|Autobiographer|   false|    11|
|    3|2012-01-17T21:11:...|        null|  7|Autobiographer|   false|    20|
|    3|2012-01-17T21:11:...|        null|  8|Autobiographer|   false|    22|
|    3|2012-01-17T21:11:...|        null|  9|Autobiographer|   false|    29|
|    3|2012-01-17T21:11:...|        null| 10|       Student|   false|     5|

#### LOAD AND SAVE COMMENTS.XML DATASET TO PARQUET FORMAT

In [42]:
schema = StructType([ \
    StructField("_ContentLicense", StringType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_PostId", LongType(), True), \
    StructField("_Score", LongType(), True), \
    StructField("_Text", StringType(), True), \
    StructField("_UserDisplayName", StringType(), True), \
    StructField("_UserId", LongType(), True)        
])

commentsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='comments') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Comments.xml')

commentsDF = commentsDF \
                .withColumnRenamed("_ContentLicense", "ContentLicense") \
                .withColumnRenamed("_CreationDate", "CreationDate") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_PostId", "PostId") \
                .withColumnRenamed("_Score", "Score") \
                .withColumnRenamed("_Text", "Text") \
                .withColumnRenamed("_UserDisplayName", "UserDisplayName") \
                .withColumnRenamed("_UserId", "UserId") \

commentsDF = commentsDF \
  .select(["Id", "PostId", "UserId", "CreationDate", "ContentLicense", \
           "UserDisplayName", "Score", "Text"])

commentsDF.printSchema()

commentsDF \
    .write \
    .partitionBy("ContentLicense") \
    .mode("overwrite") \
    .parquet("/content/output/comments.parquet")

root
 |-- Id: long (nullable = true)
 |-- PostId: long (nullable = true)
 |-- UserId: long (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- Score: long (nullable = true)
 |-- Text: string (nullable = true)



#### LOAD AND SAVE POSTHISTORY.XML DATASET TO PARQUET FORMAT

In [12]:
postHistoryDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='posthistory') \
          .options(rowTag='row') \
          .load('/content/input/PostHistory.xml')

postHistoryDF.show()  

+--------+---------------+--------------------+---+------------------+-------+--------------------+--------------------+----------------+-------+
|_Comment|_ContentLicense|       _CreationDate|_Id|_PostHistoryTypeId|_PostId|       _RevisionGUID|               _Text|_UserDisplayName|_UserId|
+--------+---------------+--------------------+---+------------------+-------+--------------------+--------------------+----------------+-------+
|    null|   CC BY-SA 3.0|2012-01-17T21:03:...|  1|                 2|      1|1bdde89b-1a24-4e8...|From the front en...|            null|      5|
|    null|   CC BY-SA 3.0|2012-01-17T21:03:...|  2|                 1|      1|1bdde89b-1a24-4e8...|How is \[Invisibl...|            null|      5|
|    null|   CC BY-SA 3.0|2012-01-17T21:03:...|  3|                 3|      1|1bdde89b-1a24-4e8...|         <front-end>|            null|      5|
|    null|   CC BY-SA 3.0|2012-01-17T21:10:...|  4|                 2|      2|608fbddf-c71d-4c2...|`Cases`, `Select`...|    

#### LOAD AND SAVE TAGS.XML DATASET TO PARQUET FORMAT

In [21]:
tagsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='tags') \
          .options(rowTag='row') \
          .load('/content/input/Tags.xml')

tagsDF.count()

718

#### LOAD AND SAVE USERS.XML DATASET TO PARQUET FORMAT

In [16]:
usersDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='Users') \
          .options(rowTag='row') \
          .load('/content/input/Users.xml')

usersDF.show() 

+--------------------+----------+--------------------+----------------+----------+---+--------------------+--------------------+--------------------+-----------+--------+------+--------------------+
|            _AboutMe|_AccountId|       _CreationDate|    _DisplayName|_DownVotes|_Id|     _LastAccessDate|           _Location|    _ProfileImageUrl|_Reputation|_UpVotes|_Views|         _WebsiteUrl|
+--------------------+----------+--------------------+----------------+----------+---+--------------------+--------------------+--------------------+-----------+--------+------+--------------------+
|<p>Hi, I'm not re...|        -1|2012-01-17T19:45:...|       Community|      5535| -1|2012-01-17T19:45:...|  on the server farm|                null|          1|   16546|   224|http://meta.stack...|
|<p>Dev #2 who hel...|         2|2012-01-17T20:50:...|    Geoff Dalgas|         0|  1|2016-11-18T21:48:...|       Corvallis, OR|https://i.stack.i...|        101|       2|  1743|http://stackoverf...|
|<p>I

#### LOAD AND SAVE VOTES.XML DATASET TO PARQUET FORMAT

In [18]:
votesDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='votes') \
          .options(rowTag='row') \
          .load('/content/input/Votes.xml')

votesDF.show(n=1000) 

+-------------+--------------------+----+-------+-------+-----------+
|_BountyAmount|       _CreationDate| _Id|_PostId|_UserId|_VoteTypeId|
+-------------+--------------------+----+-------+-------+-----------+
|         null|2012-01-17T00:00:...|   1|      1|   null|          2|
|         null|2012-01-17T00:00:...|   2|      1|   null|          2|
|         null|2012-01-17T00:00:...|   3|      1|   null|          2|
|         null|2012-01-17T00:00:...|   4|      1|   null|          2|
|         null|2012-01-17T00:00:...|   5|      1|   null|          2|
|         null|2012-01-17T00:00:...|   6|      2|   null|          2|
|         null|2012-01-17T00:00:...|   7|      2|   null|          2|
|         null|2012-01-17T00:00:...|   8|      2|   null|          2|
|         null|2012-01-17T00:00:...|   9|      2|   null|          2|
|         null|2012-01-17T00:00:...|  10|      1|   null|          2|
|         null|2012-01-17T00:00:...|  11|      2|   null|          2|
|         null|2012-